# 고속 옵티마이저
지금까지 학습 속도를 높이는 네 가지 방법을 보았음.
- 연결 가중치에 좋은 초기화 전략 적용하기
- 좋은 활섬함수 사용하기
- 배치 정규화 사용하기
- 사전학습된 네트워크의 일부 재사용하기

학습 속도를 크게 높일 수 있는 도 다른 방법으로는 표준적인 경사 하강법 옵티마이저가 아닌 더 빠른 옵티마이저를 사용하는 것임.  
이 절에서는 가장 인기있는 옵티마이저를 소개함
>- **모멘텀 최적화** (momentum optimization)
>- **네스테로프 가속 경사** (Nesterov accelerated gradient)
>- **AdaGrad**
>- **RMSProp**
>- **Adam**
>- **Nadam**

---
## 모멘텀 최적화
표준적인 경사 하강법은 경사면을 따라 일정한 크기의 스텝으로 조금씩 내려감. 맨 아래 도착하는 데 시간이 오래 걸림.  
**모멘텀 최적화** 는 이전 그래디언트가 얼마였는지를 중요하게 고려함.  
> 매 반복에서 현재 그레디언트(학습률 곱한 후)를 **모멘텀 벡터**에 더하고 이 값을 빼는 방식으로 가중치를 갱신

다시 말해 그레디언트를 가속도로 사용함.  
그리고 모멘텀이 매우 커지는 것을 막기 위해 알고리즘에 **모멘텀** 이라는 하이퍼파라미터 **베타** 가 등장함.  
> 베타는 0 ~ 1사이의 값을 가지며 0은 높은 마찰 저항, 1은 마찰 저항 없음을 뜻함.  
일반적으로 모멘텀 값은 0.9로 설정

> **모멘텀 최적화는 일반 경사 하강법보다 거의 항상 더 빠름.**  

In [1]:
from tensorflow import keras

In [2]:
opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)

케라스에서 모멘텀을 사용하는 예시

---
## 네스테로프 가속 경사
**NAG (Nesterov accelerated gradient**  
모멘텀 최적화와는 달리 현재 위치의 세타를 기준으로 하는 것이 아니라  
모멘텀 방향으로 조금 앞선 시점에서의 세타를 기준으로 그레디언트를 계산하는 알고리즘
>모멘텀 최적화의 한 변종으로 **기본 모멘텀 최적화보다 거의 항상 더 빠름**.  

In [3]:
opt = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

---
## AdaGrad
손실 함수가 타원형태일때 경사 하강법은 전역 최적점 방향으로 곧장 향하는 것이 아니라 가장 가파른 경사를 따라 내려가기 때문에 수렴을 늦게 함. 만약 알고리즘이 이를 일찍 감지하고 전역 최적점 쪽으로 좀 더 정확한 방향을 잡았다면 좋았을 것.  
>AdaGrad는 가장 가파른 차원을 따라 그레디언트 벡터의 스케일을 감소시켜 이 문제를 해결

> 이 알고리즘은 **학습률을 감소시키지만 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소됨**  
이를 **적응적 학습률** 이라고 부르며 전역 최적점 방향으로 더 곧장 가도록 갱신하는데 도움이 됨.  

그런데 AdaGrad는 2차 방정식같은 간단한 문제에서는 잘 작동하지만 신경망을 학습할 때는 너무 일찍 멈추는 경우가 있음.  
학습률이 너무 감소되어 전역 최적점에 도달하기 전에 알고리즘이 완전히 멈춘다고 함.

> 따라서 케라스에는 Adagrad 옵티마이저가 있지만 **심층 신경망에는 사용하지 말아야 함**

---
## RMSProp
AdaGrad는 너무 빨리 느려져서 전역 최적점에 수렴하지 못하는 위험이 있음.  
>**RMSProp**은 가장 최근 반복에서 비롯된 그레디언트만 누적함으로써 이 문제를 해결함.
새로운 하이퍼파라미터 **감쇠율(베타)** 가 필요함. 기본값은 보통 0.9로 잘 작동하므로 튜닝할 필요 없음.

In [5]:
opt = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

케라스에서 RMSprop을 사용하는 예시. **rho**가 감쇠율에 해당함.  
> 아주 간단한 문제를 제외하고는 **AdaGrad** 보다 언제나 성능이 훨씬 더 좋다고 함.

---
## Adam
적응적 모멘트 추정**Adaptive moment estimation**을 의미하는 **Adam**은 모멘텀 최적화와 RMSProp의 아이디러를 합친 것임.  
>**모멘텀 감쇠 하이퍼파라미터 (베타1)** 는 보통 0.9로 초기화  
**스케일 감쇠 하이퍼파라미터 (베타2)** 는 보통 0.999로 초기화.  
**엡실론** 은 보통 10의 -7승같은 아주 작은 수로 초기화

In [6]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

> Adam은 (AdaGrad나 RMSProp처럼) 적응적 학습률 알고리즘이기 때문에 학습률 파라미터를 튜닝할 필요가 적음.  
기본값 0.001을 일반적으로 사용하므로 경사 하강법보다 Adam이 사용하기 더 쉬움.

---
## AdaMax
> 실전에서 Adam보다 더 안정적이라고 함.  
하지만 데이터셋에 따라 다르고 일반적으로 Adam의 성능이 더 낫다고 함.  
어떤 작업에서 Adam이 잘 작동하지 않을 경우 시도해볼만 한 것

---
## Nadam
Adam옵티마이저에 네스테로프 기법을 더한 것.  
> 종종 Adam보다 조금 더 바르게 수렴함.  
일반적으로 Nadam이 Adam보다 성능이 좋았지만 RMSProp이 나을 때도 있었다고 함.

---
## 옵티마이저 요약
|**클래스**|**수렴 속도**|**수렴 품질**|  
|---|---|---|  
|**SGD**|느림|좋음|  
|**SGD (+ momentum)**|중간|좋음|  
|**SGD (+ momentum, + nesterov)**| 중간| 좋음|  
|**Adagrad**| 빠름| 안좋음(너무 일찍 멈춤)|  
|**RMSProp**|빠름|중간 or 좋음|  
|**Adam**|빠름|중간 or 좋음|  
|**Nadam**|빠름|중간 or 좋음|  
|**AdaMax**|빠름|중간 or 좋음|  

---
## 학습률 스케쥴링
- **학습률이 너무 많이 클 때** : 발산  
- **학습률이 작을 때** : 수렴 안 됨(느림)  
- **학습률이 클 때** : 차선의 수렴
- **학습률이 적절할 때** : 최선의 수렴
> **즉 큰 학습률로 시작하여 점점 작아지면 완벽!**

일정한 학습률로 사용하기보다는 큰 학습률(가능한)로 시작하고 학습 속도가 느려질 쯤 학습률을 낮추면 훨씬 더 발리 좋은 수렴을 할 수 있음.  
이런 전략을 **학습 스케줄링**이라고 함

다음이 가장 널리 사용되는 학습 스케줄 방법이라고 함
> - **거듭제곱 기반 스케줄링**  
>- **지수 기반 스케줄링**  
>- **구간별 고정 스케줄링** 
>- **성능 기반 스케줄링**  
>- **1 사이클 스케줄링**

> 2013년 앤드루 시니어의 논문에서 모멘텀 최적화를 사용한 학습에서 상기 스케줄링의 성능을 비교해봤을 때  
성능 기반 스케줄링과 지수 기반 스케줄링이 둘 다 잘 작동했고  
그 중에서 튜닝이 수비고 최적점에 조금 더 빨리 수렴하는 **지수 기반 스케줄링**이 선호된다고 결론을 냈다고 함.  

>**1 사이클 스케줄링**이 조금 더 좋다고 함(???)

In [7]:
opt = keras.optimizers.SGD(lr=0.01, decay=1e-4)

**거듭 제곱 기반 스케줄링**의 구현 예시. 제일 쉬움  
**decay**는 s(학습률을 나누기 위해 수행할 스텝 수)의 역수임.

---
## 지수 기반 스케줄링 코드 구현

In [9]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

In [10]:
exp_decay_fn = exponential_decay(lr0=0.01, s=20)

In [13]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay_fn)
'''hist = model.fit(x_train, y_train, 
                callbacks=[lr_scheduler])'''

'hist = model.fit(x_train, y_train, \n                callbacks=[lr_scheduler])'

**LearningRateScheduler**는 에포크를 시작할 때마다 옵티마이저의 **learning_rate** 속성을 업데이트함.  
에포크마다 한 번씩 스케줄을 업데이트해도 보통 충분하다고 함.  
만약 에포크마다 스텝이 많다면 스텝마다 학습률을 업데이트하는 것이 좋을 수도 있음.

In [14]:
def exponential_decay_fn(epoch, lr):
    return lr * 0.1**(1 / 20)

스케줄 함수는 두 번째 매개변수로 현재 학습률을 받을 수도 있음.  
이 구현은 이전과 달리 옵티마이저의 초기 학습률에만 의존하므로 이를 적절히 설정해야 함.

모델을 저장할 때 옵티마이저와 학습률도 함께 저장이 됨. 새로운 스케줄 함수를 사용할 때도 아무 문제없이 학습된 모델을 로드하여 중지된 지점부터 학습을 계속 진행할 수 있음.  
하지만 스케줄 함수가 **epoch**매개변수를 이용하면 문제가 복잡해짐. 에포크는 저장되지 않고 fit메서드를 호출할 때마다 0으로 초기화되기 때문.  
중지된 지점부터 모델 학습을 이어가려 한다면 매우 큰 학습률로 시작해서 모델의 가중치를 망가뜨릴 가능성이 높음.  
>해결할 방법은 특정 epoch에서 시작하도록 fit메서드의 **initial_epoch** 매개변수를 수동으로 지정하는 것.

---
## 구간별 스케줄링 코드 구현

In [15]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

---
## 성능 기반 스케줄링 코드 구현


In [16]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

**ReduceLROnPlateau** 콜백을 사용.  
>상기 콜백을 fit메서드에 전달하면 최상의 검증 손실이 다섯 번의 연속적인 에포크 동안 향상되지 않을 때마다 학습률에 0.5를 곱함.  

다른 옵션도 가능하니 온라인 문서 참고

---
## kears.optimizers.schedules
케라스가 기본적으로 제공하는 스케줄링 방법 중 하나를 사용해 학습률을 정의하고 이 학습률을 옵티마이저에 전달.  
>이렇게 하면 에포크가 아니라 매 스텝마다 학습률을 업데이트함.  

예를 들어 다음은 앞서 정의한 **exponential_decay_fn**과 동일한 지수 기반 스케줄링을 구현하는 방법임

In [21]:
import numpy as np
x_train = np.arange(0, 100)
s = 20 * len(x_train) // 32             # 배치 사이즈가 32일 때, 20번 에포크에 담긴 전체 스텝수
decaying_lr = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
opt = keras.optimizers.SGD(learning_rate=decaying_lr)

In [26]:
for i in range(0, 10):
    print(decaying_lr(step=i))

tf.Tensor(0.01, shape=(), dtype=float32)
tf.Tensor(0.009635427, shape=(), dtype=float32)
tf.Tensor(0.009284145, shape=(), dtype=float32)
tf.Tensor(0.008945671, shape=(), dtype=float32)
tf.Tensor(0.008619536, shape=(), dtype=float32)
tf.Tensor(0.008305291, shape=(), dtype=float32)
tf.Tensor(0.008002502, shape=(), dtype=float32)
tf.Tensor(0.0077107525, shape=(), dtype=float32)
tf.Tensor(0.0074296393, shape=(), dtype=float32)
tf.Tensor(0.0071587753, shape=(), dtype=float32)
